## Previsão de insatisfação de cliente em compras de e-commerce a fim de permitir ações proativas de forma preventiva
 - Projeto de Disciplina - Redes neurais com TensorFlow [25E3_2]
 - Aluna: Rosana Ribeiro Lima
 - Dataset: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_orders_dataset.csv


-------------------------------
### Pitch de negócio

##### Problema:

Com frequência, empresas de e-commerce perdem a fidelização de clientes em razão de instatisfação, seja por atrasos, falhas na entrega, e outros motivos. Isso gera inevitavelmente perdas na receita da empresa, além de impactar a imagem que as pessoas tem sobre a confiabilidade da instituição. 

Saber prever a insatisfação do cliente (que pode ser identificada por avaliações negativas) com base em dados do pedido antes da entrega ser finalizada poderia permitir ações proativas, como envio de brindes pelos vendedores, antecipação de problemas ou melhoria logística, no caso de atrasos.

##### Solução proposta:

A solução proposta visa analisar o dataset em questão e utilizá-lo como base para criar um modelo de redes neurais que permita identificar as principais causas que possam ser identificadas previamente como motivo para eventuais avaliações ruins de um pedido.

Além disso, caso seja identificado que o atraso na entrega tem um peso importante na avaliação ruim do pedido, a solução se propõe a identificar eventuais features que possam impactar nesse atraso, a fim de que medidas preventivas de logísticas possam ser tomadas pela empresa.

--------------------
### Features propostas

Com base no cruzamento de dados das tabelas disponíveis no dataset em questão, as features propostas para utilização são:

OK:

- dia da compra
- tempo da compra até a aprovacao do pagamento
- tempo da confirmacao da compra até a entrega
- diferenca entre tempo previsto para a entrega e tempo real de entrega
- quantidade de itens no pedido (quantidade comprada)
- price
- freight_value (frete)
- estado_do_comprador

TODO:

- payment_type
- payment_value
- estado_do_vendedor
- tamanho_da_descricao_do_produto (número de caracteres)
- categoria_do_produto
- número_de_fotos_do_produto
- peso_do_produto
- altura_do_produto


Originalmente, os dados estão distribuídos em diversas tabelas. Então teremos algumas tarefas a serem cumpridas:
1. juntar as features desejadas em um único dataframe
2. analisar os dados e tratar eventuais dados repetidos ou nulos
3. transformar variáveis categóricas, usando one hot encoding, label encoding ou embedding + SHAP
4. verificar o histograma de distribuição das features a fim de, se necessário, realizar normalização, ou escalonamento, padronização, padronização robusta... (obs.: essa etapa deve ocorrer após a separação entre treino e teste, para que nao haja interferência dos dados de teste nos dados de treino)
5. adicionar os dados na rede

--------------------

#### Etapa 1. Juntando as features em um único dataframe

##### 1.1 Obtendo as features de data e tempo

Vamos iniciar pela obtenção das seguintes features:
- dia da compra
- tempo da compra até a aprovacao do pagamento
- tempo da confirmacao da compra até a entrega
- diferenca entre tempo previsto para a entrega e tempo real de entrega

Tendo em vista que elas já se encontram na tabela de pedidos (olist_orders_dataset.csv).

In [33]:
import os
import pandas as pd

path = "../data/raw/"
df = pd.read_csv(os.path.join(path, "olist_orders_dataset.csv"))

# Remover coluna desnecessária
df = df.drop(columns=["order_status"])

# Converter colunas de data
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

# Filtrar linhas com datas válidas e criar cópia
df_filtrado = df.dropna(subset=['order_estimated_delivery_date', 'order_delivered_customer_date','order_purchase_timestamp','order_approved_at']).copy()

# Criar colunas numéricas com timestamps
df_filtrado['timestamp_entrega_estimada'] = df_filtrado['order_estimated_delivery_date'].apply(lambda x: int(x.timestamp()))
df_filtrado['timestamp_entrega_real'] = df_filtrado['order_delivered_customer_date'].apply(lambda x: int(x.timestamp()))
df_filtrado['timestamp_da_compra'] = df_filtrado['order_purchase_timestamp'].apply(lambda x: int(x.timestamp()))
df_filtrado['timestamp_da_aprovacao'] = df_filtrado['order_approved_at'].apply(lambda x: int(x.timestamp()))

# Criar colunas das features relacionadas a tempo:
df_filtrado['diferenca_entre_tempo_estimado_e_real_entrega'] = (
    df_filtrado['timestamp_entrega_estimada'] - df_filtrado['timestamp_entrega_real']
)
df_filtrado['tempo_da_compra_ate_a_aprovacao'] = df_filtrado['timestamp_da_aprovacao'] - df_filtrado['timestamp_da_compra']
df_filtrado['tempo_da_aprovacao_ate_a_entrega'] = df_filtrado['timestamp_entrega_real'] - df_filtrado['timestamp_da_aprovacao'] 



# Removendo as colunas de data que não serão mais utilizadas:
df_filtrado = df_filtrado.drop(columns=["order_estimated_delivery_date", "order_delivered_customer_date", "order_purchase_timestamp","order_approved_at",
                                        "timestamp_entrega_estimada","timestamp_entrega_real", "timestamp_da_aprovacao","order_delivered_carrier_date"])

# Visualizar
df_filtrado.head(7)


,order_id,customer_id,timestamp_da_compra,diferenca_entre_tempo_estimado_e_real_entrega,tempo_da_compra_ate_a_aprovacao,tempo_da_aprovacao_ate_a_entrega
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,1506941793,614087,642,728278
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,1532464897,462735,110570,1080198
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,1533717529,1490011,994,810666
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,1511033286,1121478,1073,1140163
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,1518556719,798178,3710,244593
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,1499637425,478925,788,1428462
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,1494940230,990249,701,862420


Já temos portanto quatro das features que desejamos. Vamos agora utilizar o id do pedido (order_id) e o id do cliente (customer_id) para fazer um cruzamento dos dados de df_filtrado com os dados de outras tabela.

Um detalhe importante: tendo em vista que, no dataset, a review é por pedido, mas o mesmo pedido pode conter produtos de ids diferentes, a fim de podermos analisar o impacto de dados do produto na satisfação do cliente, vamos considerar apenas os pedidos que tenham um único id de produto associado a ele. Para isso, vamos realizar uma filtragem na tabela presente em olist_order_items_dataset.csv.

In [49]:
df_customers = pd.read_csv(os.path.join(path, "olist_customers_dataset.csv"))
df_order_items = pd.read_csv(os.path.join(path, "olist_order_items_dataset.csv"))


# Selecionar apenas os pedidos com **um único** id de produto
produtos_por_pedido = df_order_items.groupby('order_id')['product_id'].nunique()
pedidos_unico_produto = produtos_por_pedido[produtos_por_pedido == 1].index
df_order_items_filtrado = df_order_items[df_order_items['order_id'].isin(pedidos_unico_produto)]

# Filtrar para que o order_item_id não seja motivo de duplicação dos dados caso a pessoa tenha comprado mais de um produto com mesmo id_product no mesmo pedido.
# Para isso, vamos utilizar apenas a linha de maior valor do order_item_id, a fim de que ela indique a quantidade de produtos comprados.
idx = df_order_items_filtrado.groupby(['order_id', 'product_id'])['order_item_id'].idxmax()
df_order_items_unico = df_order_items_filtrado.loc[idx]
df_order_items_unico.rename(columns={'order_item_id': 'quantidade_comprada'}, inplace=True)

# Juntar orders + customers (chave: customer_id)
df_merged_1 = pd.merge(df_filtrado, df_customers, on='customer_id', how='inner')
df_merged_2 = pd.merge(df_merged_1, df_order_items_unico, on='order_id', how='inner')

# Confirmando que não há order_ids ou customer_ids duplicados após os merges
df_merged_2['order_id'].duplicated().any()
df_merged_2['customer_id'].duplicated().any()

df_merged_2


,order_id,customer_id,timestamp_da_compra,diferenca_entre_tempo_estimado_e_real_entrega,tempo_da_compra_ate_a_aprovacao,tempo_da_aprovacao_ate_a_entrega,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,quantidade_comprada,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,1506941793,614087,642,728278,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,1532464897,462735,110570,1080198,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,1533717529,1490011,994,810666,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,1511033286,1121478,1073,1140163,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,1518556719,798178,3710,244593,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93259,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,1489053245,895919,0,710036,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,1,ac35486adb7b02598c182c2ff2e05254,e24fc9fcd865784fb25705606fe3dfe7,2017-03-15 09:54:05,72.00,13.08
93260,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,1517921938,109324,699,1916839,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
93261,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,1503845203,477343,1053,2146801,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,1,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
93262,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,1515446907,1729626,474,1475793,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,2,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59


### 2. Juntando os dados em um único dataframe


#### 2.1 Descrevendo as variáveis e os tipos

Vamos primeiro fazer algumas análises dos dados:

In [2]:
# df.info()

In [3]:
# df_filtered['Asthma_Status'].value_counts()

## Referências

- OLIST. Brazilian E-Commerce Public Dataset by Olist. Kaggle, 2018. Disponível em: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce. Acesso em: 31 jul. 2025.